#### BIBLIOTECAS

In [503]:
import pandas as pd
import numpy as np
import random
import folium
import datetime

#### CARREGAR ARQUIVO

In [504]:
df = pd.read_csv('trips_data.csv', sep = ',')

In [505]:
display(df.head(2))
display(df.shape)

,City,Product Type,Trip or Order Status,Request Time,Begin Trip Time,Begin Trip Lat,Begin Trip Lng,Begin Trip Address,Dropoff Time,Dropoff Lat,Dropoff Lng,Dropoff Address,Distance (miles),Fare Amount,Fare Currency
0,801,UberX,COMPLETED,2022-02-09 09:47:27 +0000 UTC,2022-02-09 09:53:27 +0000 UTC,-3.759715,-38.57086,"Rua Alagoas, 1505 - Demócrito Rocha - Fortalez...",2022-02-09 10:14:07 +0000 UTC,-3.754633,-38.628765,"R. Araré, 1777 - Parque Guadalajara - Caucaia ...",6.092459,18.61,BRL
1,801,UberX,COMPLETED,2022-02-07 10:19:04 +0000 UTC,2022-02-07 10:25:39 +0000 UTC,-3.759732,-38.57078,"Rua Alagoas, 1505 - Demócrito Rocha - Fortalez...",2022-02-07 10:45:36 +0000 UTC,-3.754878,-38.628490,"R. Araré, 1777 - Parque Guadalajara - Caucaia ...",6.041353,32.92,BRL


(866, 15)

#### PREPARAÇÃO DOS DADOS

##### LIMPEZA DOS DADOS

In [506]:
list(df.columns)

['City',
 'Product Type',
 'Trip or Order Status',
 'Request Time',
 'Begin Trip Time',
 'Begin Trip Lat',
 'Begin Trip Lng',
 'Begin Trip Address',
 'Dropoff Time',
 'Dropoff Lat',
 'Dropoff Lng',
 'Dropoff Address',
 'Distance (miles)',
 'Fare Amount',
 'Fare Currency']

In [507]:
# FILTRAR COLUNAS DESEJADAS
df = df.drop(['City',
         'Begin Trip Address',
         'Dropoff Address',
         'Fare Currency'], axis = 1)
df.head(3)

,Product Type,Trip or Order Status,Request Time,Begin Trip Time,Begin Trip Lat,Begin Trip Lng,Dropoff Time,Dropoff Lat,Dropoff Lng,Distance (miles),Fare Amount
0,UberX,COMPLETED,2022-02-09 09:47:27 +0000 UTC,2022-02-09 09:53:27 +0000 UTC,-3.759715,-38.570860,2022-02-09 10:14:07 +0000 UTC,-3.754633,-38.628765,6.092459,18.61
1,UberX,COMPLETED,2022-02-07 10:19:04 +0000 UTC,2022-02-07 10:25:39 +0000 UTC,-3.759732,-38.570780,2022-02-07 10:45:36 +0000 UTC,-3.754878,-38.628490,6.041353,32.92
2,UberX,COMPLETED,2022-02-04 21:14:35 +0000 UTC,2022-02-04 21:23:13 +0000 UTC,-3.754990,-38.556026,2022-02-04 21:30:56 +0000 UTC,-3.759685,-38.570805,1.367133,8.92


In [508]:
# TIPOS DE CORRIDA
df['Trip or Order Status'].unique()

array(['COMPLETED', 'UNFULFILLED', 'DRIVER_CANCELED', 'CANCELED'],
      dtype=object)

In [509]:
# UNFULLFILLED SÃO CORRIDAS CANCELADAS PELO SISTEMA, ANTES DE ENCOTRAR UM MOTORISTA
# ESSAS LINHAS SERÃO EXCLUIDAS
display(df[df['Trip or Order Status'] == 'UNFULFILLED'].head(3))
display(df[df['Trip or Order Status'] == 'UNFULFILLED'].shape)

,Product Type,Trip or Order Status,Request Time,Begin Trip Time,Begin Trip Lat,Begin Trip Lng,Dropoff Time,Dropoff Lat,Dropoff Lng,Distance (miles),Fare Amount
6,NaN,UNFULFILLED,2022-02-03 10:27:18 +0000 UTC,1970-01-01 00:00:00 +0000 UTC,NaN,NaN,1970-01-01 00:00:00 +0000 UTC,NaN,NaN,0.0,0.0
7,NaN,UNFULFILLED,2022-02-03 10:22:44 +0000 UTC,1970-01-01 00:00:00 +0000 UTC,-3.762452,-38.575176,1970-01-01 00:00:00 +0000 UTC,-3.76141,-38.574494,0.0,0.0
87,NaN,UNFULFILLED,2021-10-03 01:20:49 +0000 UTC,1970-01-01 00:00:00 +0000 UTC,NaN,NaN,1970-01-01 00:00:00 +0000 UTC,NaN,NaN,0.0,0.0


(18, 11)

In [510]:
# VARIAVEL COM LINHAS PARA EXCLUIR
excluir_linhas = df[df['Trip or Order Status'] == 'UNFULFILLED'].index
# EXCLUIR LINHAS
df = df.drop(excluir_linhas, axis = 0)
df.shape

(848, 11)

In [511]:
df['Product Type'].value_counts()

UberX                   634
uberX                   164
VIP                      21
UberX Promo               9
UberEATS Marketplace      7
SELECT                    5
Juntos: MATCHED           3
Moto                      2
Flash                     1
Comfort                   1
Juntos                    1
Name: Product Type, dtype: int64

In [512]:
# EXCLUIR O PRODUTO 'UBER EATS' E 'FLASH', QUEREMOS APENAS O SERVIÇO DE VIAGENS/CORRIDAS
# EXCLUIR O PRODUTO 'UBER EATS'
display(df['Product Type'].unique())
excluir_linhas = df[df['Product Type'] == 'UberEATS Marketplace'].index
print('Qtd linhas para exlcuir: ', len(excluir_linhas))
df = df.drop(excluir_linhas, axis = 0)
display(df.shape)

# EXCLUIR O PRODUTO 'FLASH' (ENTREGAS)
display(df['Product Type'].unique())
excluir_linhas = df[df['Product Type'] == 'Flash'].index
print('Qtd linhas para exlcuir: ', len(excluir_linhas))
df = df.drop(excluir_linhas, axis = 0)
df.shape

array(['UberX', 'Flash', 'VIP', 'UberX Promo', 'Moto', 'Comfort',
       'UberEATS Marketplace', 'Juntos: MATCHED', 'Juntos', 'uberX',
       'SELECT'], dtype=object)

Qtd linhas para exlcuir:  7


(841, 11)

array(['UberX', 'Flash', 'VIP', 'UberX Promo', 'Moto', 'Comfort',
       'Juntos: MATCHED', 'Juntos', 'uberX', 'SELECT'], dtype=object)

Qtd linhas para exlcuir:  1


(840, 11)

In [513]:
# TIPOS DE PRODUTOS
df['Product Type'].value_counts()

UberX              634
uberX              164
VIP                 21
UberX Promo          9
SELECT               5
Juntos: MATCHED      3
Moto                 2
Comfort              1
Juntos               1
Name: Product Type, dtype: int64

In [514]:
# EXISTEM PRODUTOS IGUAIS DESCRITOS DE FORMA DIFERENTE, VAMOS UNIFICAR RENOMEANDO
df['Product Type'] = df['Product Type'].replace('uberX', 'UberX')
df['Product Type'] = df['Product Type'].replace('UberX Promo', 'UberX')
df['Product Type'] = df['Product Type'].replace('Juntos: MATCHED', 'Juntos')
df['Product Type'].value_counts()

UberX      807
VIP         21
SELECT       5
Juntos       4
Moto         2
Comfort      1
Name: Product Type, dtype: int64

In [515]:
# AINDA TEMOS LINHAS EM BRANCO NAS VARIAVEIS DE LOCALIDADE
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 840 entries, 0 to 865
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Product Type          840 non-null    object 
 1   Trip or Order Status  840 non-null    object 
 2   Request Time          840 non-null    object 
 3   Begin Trip Time       840 non-null    object 
 4   Begin Trip Lat        819 non-null    float64
 5   Begin Trip Lng        819 non-null    float64
 6   Dropoff Time          840 non-null    object 
 7   Dropoff Lat           819 non-null    float64
 8   Dropoff Lng           819 non-null    float64
 9   Distance (miles)      840 non-null    float64
 10  Fare Amount           840 non-null    float64
dtypes: float64(6), object(5)
memory usage: 78.8+ KB


In [516]:
# VAMOS ANALISAR SE AS LINHAS EM BRANCO SÃO DE CORRIDAS CANCELADAS
display(df[df['Begin Trip Lat'].isna()]['Trip or Order Status'].unique())
display(df[df['Begin Trip Lng'].isna()]['Trip or Order Status'].unique())
display(df[df['Dropoff Lat'].isna()]['Trip or Order Status'].unique())
display(df[df['Dropoff Lng'].isna()]['Trip or Order Status'].unique())

# OU SEJA, OS DADOS FALTANTES SÃO DE CORRIDAS CANCELADAS
# NÃO FAREMOS A EXCLUSÃO DESSAS LINHAS POIS QUEREMOS USAR OS DADOS DE CORRIDAS CANCELADAS


array(['CANCELED', 'DRIVER_CANCELED'], dtype=object)

array(['CANCELED', 'DRIVER_CANCELED'], dtype=object)

array(['CANCELED', 'DRIVER_CANCELED'], dtype=object)

array(['CANCELED', 'DRIVER_CANCELED'], dtype=object)

##### FORMATAÇÃO DOS DADOS

In [517]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 840 entries, 0 to 865
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Product Type          840 non-null    object 
 1   Trip or Order Status  840 non-null    object 
 2   Request Time          840 non-null    object 
 3   Begin Trip Time       840 non-null    object 
 4   Begin Trip Lat        819 non-null    float64
 5   Begin Trip Lng        819 non-null    float64
 6   Dropoff Time          840 non-null    object 
 7   Dropoff Lat           819 non-null    float64
 8   Dropoff Lng           819 non-null    float64
 9   Distance (miles)      840 non-null    float64
 10  Fare Amount           840 non-null    float64
dtypes: float64(6), object(5)
memory usage: 78.8+ KB


In [518]:
'''
# EXTRAIR A DATA E HORA DO EMBARQUE E DESEMBARQUE E FORMATAR

# EMBARQUE
df['Begin Data'] = df['Begin Trip Time'].apply(lambda x: pd.to_datetime(x[:10], format = '%Y-%m-%d').date())
df['Begin Hora'] = df['Begin Trip Time'].apply(lambda x: pd.to_datetime(x[11:18], format = '%H:%M:%S').time())

# DESEMBARQUE
#df['Dropoff Data'] = df['Dropoff Time'].apply(lambda x: pd.to_datetime(x[:10], format = '%Y-%m-%d').date())
df['Dropoff Hora'] = df['Dropoff Time'].apply(lambda x: pd.to_datetime(x[11:18], format = '%H:%M:%S').time())

df.head(3)
'''

"\n# EXTRAIR A DATA E HORA DO EMBARQUE E DESEMBARQUE E FORMATAR\n\n# EMBARQUE\ndf['Begin Data'] = df['Begin Trip Time'].apply(lambda x: pd.to_datetime(x[:10], format = '%Y-%m-%d').date())\ndf['Begin Hora'] = df['Begin Trip Time'].apply(lambda x: pd.to_datetime(x[11:18], format = '%H:%M:%S').time())\n\n# DESEMBARQUE\n#df['Dropoff Data'] = df['Dropoff Time'].apply(lambda x: pd.to_datetime(x[:10], format = '%Y-%m-%d').date())\ndf['Dropoff Hora'] = df['Dropoff Time'].apply(lambda x: pd.to_datetime(x[11:18], format = '%H:%M:%S').time())\n\ndf.head(3)\n"

In [519]:
# EXTRAIR A DATA E HORA DO EMBARQUE E DESEMBARQUE E FORMATAR

# EMBARQUE
df['Begin Data'] = df['Begin Trip Time'].apply(lambda x: x[:10])
df['Begin Hora'] = df['Begin Trip Time'].apply(lambda x: pd.to_datetime(x[11:18], format = '%H:%M:%S').time())
df['Begin Data'] = pd.to_datetime(df['Begin Data'])

# DESEMBARQUE
#df['Dropoff Data'] = df['Dropoff Time'].apply(lambda x: pd.to_datetime(x[:10], format = '%Y-%m-%d').date())
df['Dropoff Hora'] = df['Dropoff Time'].apply(lambda x: pd.to_datetime(x[11:18], format = '%H:%M:%S').time())

df.head(3)


,Product Type,Trip or Order Status,Request Time,Begin Trip Time,Begin Trip Lat,Begin Trip Lng,Dropoff Time,Dropoff Lat,Dropoff Lng,Distance (miles),Fare Amount,Begin Data,Begin Hora,Dropoff Hora
0,UberX,COMPLETED,2022-02-09 09:47:27 +0000 UTC,2022-02-09 09:53:27 +0000 UTC,-3.759715,-38.570860,2022-02-09 10:14:07 +0000 UTC,-3.754633,-38.628765,6.092459,18.61,2022-02-09,09:53:02,10:14:00
1,UberX,COMPLETED,2022-02-07 10:19:04 +0000 UTC,2022-02-07 10:25:39 +0000 UTC,-3.759732,-38.570780,2022-02-07 10:45:36 +0000 UTC,-3.754878,-38.628490,6.041353,32.92,2022-02-07,10:25:03,10:45:03
2,UberX,COMPLETED,2022-02-04 21:14:35 +0000 UTC,2022-02-04 21:23:13 +0000 UTC,-3.754990,-38.556026,2022-02-04 21:30:56 +0000 UTC,-3.759685,-38.570805,1.367133,8.92,2022-02-04,21:23:01,21:30:05


In [520]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 840 entries, 0 to 865
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Product Type          840 non-null    object        
 1   Trip or Order Status  840 non-null    object        
 2   Request Time          840 non-null    object        
 3   Begin Trip Time       840 non-null    object        
 4   Begin Trip Lat        819 non-null    float64       
 5   Begin Trip Lng        819 non-null    float64       
 6   Dropoff Time          840 non-null    object        
 7   Dropoff Lat           819 non-null    float64       
 8   Dropoff Lng           819 non-null    float64       
 9   Distance (miles)      840 non-null    float64       
 10  Fare Amount           840 non-null    float64       
 11  Begin Data            840 non-null    datetime64[ns]
 12  Begin Hora            840 non-null    object        
 13  Dropoff Hora        

In [521]:
# AO LEVAR OS DADOS PARA ANALISE NO BI, FOI PERCECBIDO QUE AS CORRIDAS CANCELADAS TIVERAM
# O INICIO DA CORRIDA(BEGIN TRIP) PREENCHIDOS COM O ANO 1970
# PARA SOLUCIONAR ESSA QUESTÃO, IREI REPETIR A INFORMAÇÃO DE SOLICITAÇÃO DA CORRIDA (REQUEST TIME)
# NA COLUNA DE EMBARQUE (BEGIN TRIP) PARA O ANO DE 1970

In [522]:
df['ano'] = df['Begin Data'].dt.year

In [523]:
df['ano'].value_counts()

2021    240
2019    150
2018    132
2020     99
2017     91
1970     90
2022     28
2016     10
Name: ano, dtype: int64

In [524]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 840 entries, 0 to 865
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Product Type          840 non-null    object        
 1   Trip or Order Status  840 non-null    object        
 2   Request Time          840 non-null    object        
 3   Begin Trip Time       840 non-null    object        
 4   Begin Trip Lat        819 non-null    float64       
 5   Begin Trip Lng        819 non-null    float64       
 6   Dropoff Time          840 non-null    object        
 7   Dropoff Lat           819 non-null    float64       
 8   Dropoff Lng           819 non-null    float64       
 9   Distance (miles)      840 non-null    float64       
 10  Fare Amount           840 non-null    float64       
 11  Begin Data            840 non-null    datetime64[ns]
 12  Begin Hora            840 non-null    object        
 13  Dropoff Hora        

In [525]:
display(df.head(3))

,Product Type,Trip or Order Status,Request Time,Begin Trip Time,Begin Trip Lat,Begin Trip Lng,Dropoff Time,Dropoff Lat,Dropoff Lng,Distance (miles),Fare Amount,Begin Data,Begin Hora,Dropoff Hora,ano
0,UberX,COMPLETED,2022-02-09 09:47:27 +0000 UTC,2022-02-09 09:53:27 +0000 UTC,-3.759715,-38.570860,2022-02-09 10:14:07 +0000 UTC,-3.754633,-38.628765,6.092459,18.61,2022-02-09,09:53:02,10:14:00,2022
1,UberX,COMPLETED,2022-02-07 10:19:04 +0000 UTC,2022-02-07 10:25:39 +0000 UTC,-3.759732,-38.570780,2022-02-07 10:45:36 +0000 UTC,-3.754878,-38.628490,6.041353,32.92,2022-02-07,10:25:03,10:45:03,2022
2,UberX,COMPLETED,2022-02-04 21:14:35 +0000 UTC,2022-02-04 21:23:13 +0000 UTC,-3.754990,-38.556026,2022-02-04 21:30:56 +0000 UTC,-3.759685,-38.570805,1.367133,8.92,2022-02-04,21:23:01,21:30:05,2022


In [526]:
for a, r, b, i in zip(df['ano'], df['Request Time'], df['Begin Trip Time'], df['Begin Trip Time'].index):
    if a == 1970:
        df['Begin Trip Time'][i] = r
    else:
        df['Begin Trip Time'][i] = b


display(df[df['ano'] == 1970].head(3))
display(df[df['ano'] != 1970].head(3)) ;

C:\Users\MS\AppData\Local\Temp/ipykernel_8116/3193122790.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Begin Trip Time'][i] = b
C:\Users\MS\AppData\Local\Temp/ipykernel_8116/3193122790.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Begin Trip Time'][i] = r


,Product Type,Trip or Order Status,Request Time,Begin Trip Time,Begin Trip Lat,Begin Trip Lng,Dropoff Time,Dropoff Lat,Dropoff Lng,Distance (miles),Fare Amount,Begin Data,Begin Hora,Dropoff Hora,ano
8,UberX,DRIVER_CANCELED,2022-02-03 10:18:14 +0000 UTC,2022-02-03 10:18:14 +0000 UTC,-3.765989,-38.559654,1970-01-01 00:00:00 +0000 UTC,-3.766132,-38.559896,0.0,0.0,1970-01-01,00:00:00,00:00:00,1970
40,UberX,CANCELED,2021-12-14 21:24:01 +0000 UTC,2021-12-14 21:24:01 +0000 UTC,-3.759455,-38.568150,1970-01-01 00:00:00 +0000 UTC,-3.759264,-38.568851,0.0,0.0,1970-01-01,00:00:00,00:00:00,1970
43,UberX,DRIVER_CANCELED,2021-12-11 02:21:12 +0000 UTC,2021-12-11 02:21:12 +0000 UTC,-3.741371,-38.605263,1970-01-01 00:00:00 +0000 UTC,-3.754862,-38.628483,0.0,6.3,1970-01-01,00:00:00,00:00:00,1970


,Product Type,Trip or Order Status,Request Time,Begin Trip Time,Begin Trip Lat,Begin Trip Lng,Dropoff Time,Dropoff Lat,Dropoff Lng,Distance (miles),Fare Amount,Begin Data,Begin Hora,Dropoff Hora,ano
0,UberX,COMPLETED,2022-02-09 09:47:27 +0000 UTC,2022-02-09 09:53:27 +0000 UTC,-3.759715,-38.570860,2022-02-09 10:14:07 +0000 UTC,-3.754633,-38.628765,6.092459,18.61,2022-02-09,09:53:02,10:14:00,2022
1,UberX,COMPLETED,2022-02-07 10:19:04 +0000 UTC,2022-02-07 10:25:39 +0000 UTC,-3.759732,-38.570780,2022-02-07 10:45:36 +0000 UTC,-3.754878,-38.628490,6.041353,32.92,2022-02-07,10:25:03,10:45:03,2022
2,UberX,COMPLETED,2022-02-04 21:14:35 +0000 UTC,2022-02-04 21:23:13 +0000 UTC,-3.754990,-38.556026,2022-02-04 21:30:56 +0000 UTC,-3.759685,-38.570805,1.367133,8.92,2022-02-04,21:23:01,21:30:05,2022


In [527]:
# EXTRAIR A DATA E HORA DO EMBARQUE E DESEMBARQUE E FORMATAR

# EMBARQUE
df['Begin Data'] = df['Begin Trip Time'].apply(lambda x: x[:10])
df['Begin Hora'] = df['Begin Trip Time'].apply(lambda x: pd.to_datetime(x[11:18], format = '%H:%M:%S').time())
df['Begin Data'] = pd.to_datetime(df['Begin Data'])

# DESEMBARQUE
#df['Dropoff Data'] = df['Dropoff Time'].apply(lambda x: pd.to_datetime(x[:10], format = '%Y-%m-%d').date())
df['Dropoff Hora'] = df['Dropoff Time'].apply(lambda x: pd.to_datetime(x[11:18], format = '%H:%M:%S').time())

df.head(3)

df['ano'] = df['Begin Data'].dt.year
df['ano'].value_counts()

2021    263
2019    163
2018    153
2020    117
2017    102
2022     29
2016     13
Name: ano, dtype: int64

In [528]:
# EXCLUIR COLUNAS NÃO NECESSÁRIAS
excluir_colunas = ['Begin Trip Time', 'Request Time', 'Dropoff Time']
df = df.drop(excluir_colunas, axis = 1)

In [529]:
# CONVERTER MILHAS EM QUILOMETROS
# 1km é equivalente a 0,6214 milhas

df['Distance km'] = round(df['Distance (miles)'] / 0.6214, 2)
df = df.drop(['Distance (miles)'], axis = 1)
df.head(2)

,Product Type,Trip or Order Status,Begin Trip Lat,Begin Trip Lng,Dropoff Lat,Dropoff Lng,Fare Amount,Begin Data,Begin Hora,Dropoff Hora,ano,Distance km
0,UberX,COMPLETED,-3.759715,-38.57086,-3.754633,-38.628765,18.61,2022-02-09,09:53:02,10:14:00,2022,9.80
1,UberX,COMPLETED,-3.759732,-38.57078,-3.754878,-38.628490,32.92,2022-02-07,10:25:03,10:45:03,2022,9.72


In [530]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 840 entries, 0 to 865
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Product Type          840 non-null    object        
 1   Trip or Order Status  840 non-null    object        
 2   Begin Trip Lat        819 non-null    float64       
 3   Begin Trip Lng        819 non-null    float64       
 4   Dropoff Lat           819 non-null    float64       
 5   Dropoff Lng           819 non-null    float64       
 6   Fare Amount           840 non-null    float64       
 7   Begin Data            840 non-null    datetime64[ns]
 8   Begin Hora            840 non-null    object        
 9   Dropoff Hora          840 non-null    object        
 10  ano                   840 non-null    int64         
 11  Distance km           840 non-null    float64       
dtypes: datetime64[ns](1), float64(6), int64(1), object(4)
memory usage: 117.6+ KB


#### PLOTAR MAPA

In [531]:
# ALTERAR AS COORDENADAS LATITUDE E LONGITUDE
df['Lat'] = df['Begin Trip Lat'] + (random.randint(-1700, 1700) / 100000)
df['Lng'] = df['Begin Trip Lng'] + (random.randint(-1700, 1700) / 100000)

# INDICES EM BRANCO
i = df[df['Lat'].isna()].index
df.drop(i, axis=0)

# CRIAR MAPA COM POSIÇÃO INICIAL
mapa = folium.Map(location = [df['Lat'][0], df['Lng'][0]],\
                                width=800, height=550, zoom_start=12)

# plotar localização no mapa
for lat, lon in zip (df.drop(i, axis=0)['Lat'], df.drop(i, axis=0)['Lng']):
    folium.Marker([lat, lon], icon=folium.Icon(color='red')).add_to(mapa)

mapa


In [532]:
# EXCLUIR COLUNAS
excluir_colunas = ['Begin Trip Lat' , 'Begin Trip Lng', 'Dropoff Lat', 'Dropoff Lng', 'ano']
df = df.drop(excluir_colunas, axis = 1)

#### SALVAR ARQUIVO

In [533]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 840 entries, 0 to 865
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Product Type          840 non-null    object        
 1   Trip or Order Status  840 non-null    object        
 2   Fare Amount           840 non-null    float64       
 3   Begin Data            840 non-null    datetime64[ns]
 4   Begin Hora            840 non-null    object        
 5   Dropoff Hora          840 non-null    object        
 6   Distance km           840 non-null    float64       
 7   Lat                   819 non-null    float64       
 8   Lng                   819 non-null    float64       
dtypes: datetime64[ns](1), float64(4), object(4)
memory usage: 97.9+ KB


In [534]:
df.head(3)

,Product Type,Trip or Order Status,Fare Amount,Begin Data,Begin Hora,Dropoff Hora,Distance km,Lat,Lng
0,UberX,COMPLETED,18.61,2022-02-09,09:53:02,10:14:00,9.80,-3.766135,-38.577140
1,UberX,COMPLETED,32.92,2022-02-07,10:25:03,10:45:03,9.72,-3.766152,-38.577060
2,UberX,COMPLETED,8.92,2022-02-04,21:23:01,21:30:05,2.20,-3.761410,-38.562306


In [535]:
# SALVAR ARQUIVO
df.to_csv('corridasUberMS.csv', sep = ';')